# SED for Trappist-1
This notebook executes the lines of code presented in Filippazzo et al. (submitted to PASP). Below I generate and analyze an SED for the M8 dwarfs Trappist-1 and Gl 752B. Then I create a catalog of SEDs for analysis.

In [59]:
# Imports
from astropy import units as q
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()

Loading BokehJS ...

## Initialization
Import the `SED` class and create a Trappist-1 object.

In [60]:
from sedkit import SED
trap1 = SED(name='Trappist-1')

Setting parallax to (<Quantity 80.4512 mas>, <Quantity 0.1211 mas>) and distance to (<Quantity 12.43 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).
Setting radius to (<Quantity 0.12250478 solRad>, <Quantity 0.00153489 solRad>)
Setting spectral_type to (68.0, 0.5, 'V', None, None).
1 record for Trappist-1 found in Simbad.
Setting name to Trappist-1 and sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (346.62236833, -5.0413975)>
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)


## Photometry
Add photometry manually...

In [61]:
trap1.add_photometry('Johnson.V', 18.798, 0.082, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.R', 16.466, 0.065, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.I', 14.024, 0.115, ref='Costa et al. (2006)')

...and retrieve some from Simbad.

In [62]:
trap1.find_SDSS()
trap1.find_2MASS()
trap1.find_WISE()

1 record found in SDSS.
1 record found in 2MASS.
1 record found in WISE.


In [63]:
# Print the photometry table
trap1.photometry[['band', 'eff', 'app_magnitude', 'app_magnitude_unc', 'ref']].pprint()

   band     eff   app_magnitude app_magnitude_unc         ref        
             um                                                      
--------- ------- ------------- ----------------- -------------------
   SDSS.u 0.35949          20.7             0.091        V/147/sdss12
   SDSS.g 0.46404         19.17             0.012        V/147/sdss12
Johnson.V 0.54661          18.8             0.082 Costa et al. (2006)
   SDSS.r 0.61223          18.5             0.009        V/147/sdss12
Cousins.R  0.6358         16.47             0.065 Costa et al. (2006)
   SDSS.i 0.74395         18.23              0.01        V/147/sdss12
Cousins.I 0.78292         14.02             0.115 Costa et al. (2006)
   SDSS.z 0.88971          18.1             0.028        V/147/sdss12
  2MASS.J   1.235         11.35             0.022          II/246/out
  2MASS.H   1.662         10.72             0.021          II/246/out
 2MASS.Ks   2.159          10.3             0.023          II/246/out
  WISE.W1  3.3526   

## Spectra

In [64]:
# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Trappist-1_NIR.fits')
opt_spec = resource_filename('sedkit', 'data/Trappist-1_OPT.fits')

# Add the spectra
trap1.add_spectrum_file(nir_spec, name='NIR spectrum', wave_units=q.um, flux_units=q.W/q.m**2/q.um, ref='Filippazzo et al. (2015)')
trap1.add_spectrum_file(opt_spec, name='OPT spectrum', wave_units=q.AA, flux_units=q.erg/q.s/q.cm**2/q.AA, trim=[(0.9*q.um, 2*q.um)], ref='Cruz et al. (2003)')

# Print the table
trap1.spectra[['name', 'wave_min', 'wave_max', 'resolution', 'ref']].pprint()

    name     wave_min wave_max resolution           ref           
                um       um                                       
------------ -------- -------- ---------- ------------------------
NIR spectrum    0.653    2.566        563 Filippazzo et al. (2015)
OPT spectrum      0.6      0.9       1034       Cruz et al. (2003)


## Other Data
Set the spectral type.

In [65]:
trap1.spectral_type = 'M8'

Setting spectral_type to (68.0, 0.5, 'V', None, None).


Set the parallax.

In [66]:
trap1.parallax = 80.4512*q.mas, 0.1211*q.mas

Setting parallax to (<Quantity 80.4512 mas>, <Quantity 0.1211 mas>) and distance to (<Quantity 12.43 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).


Set the age.

In [67]:
trap1.age = 7.6*q.Gyr, 2.2*q.Gyr

Setting age to (<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>)


## Build SED

In [69]:
trap1.results.pprint(max_lines=-1)

     param               value                  unc               units    
---------------- --------------------- ---------------------- -------------
            Lbol              1.11e+29               2.28e+27       erg / s
        Lbol_sun                 -4.54                  0.009            --
            Mbol                16.097                  0.022            --
             SpT                   M8V                     --            --
         SpT_fit                    --                     --            --
            Teff                  1211                      9             K
         Teff_bb                    --                     --            --
        Teff_evo    1471.2300653594773      12.39026369168414             K
             age                   7.6                    2.2           Gyr
       bb_source                    --                     --            --
       blackbody                    --                     --            --
            

## Fit the data

In [15]:
# Fit to infer spectral type
trap1.fit_spectral_type()

Best fit:  Opt:M8V


In [16]:
# Fit to infer fundamental params
trap1.fit_modelgrid(BTSettl())

Best fit:  0.0/2700.0/5.5/0.0


## Mass, surface gravity, and Teff
Set the evolutionary model grid and infer fundamental parameters.

In [44]:
trap1.evo_model = 'DUSTY00'

In [45]:
trap1.teff_from_age(plot=True)

AttributeError: module 'sedkit.utilities' has no attribute 'errorbars'

In [42]:
trap1.logg_from_age(plot=True)

Lbol=None and age=(<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>). Both are needed to calculate the surface gravity.


In [19]:
trap1.mass_from_age(plot=True, mass_units=q.Mjup)

## Results

In [20]:
# Table of the results
trap1.results.pprint(max_lines=-1)

     param              value                  unc              units    
---------------- -------------------- --------------------- -------------
            Lbol             2.05e+30              4.22e+28       erg / s
        Lbol_sun                -3.27                 0.009            --
            Mbol               12.935                 0.012            --
             SpT                  M8V                    --            --
         SpT_fit              Opt:M8V                    --            --
            Teff                 2523                    33             K
         Teff_bb                   --                    --            --
        Teff_evo    2622.658823529412    16.662507427214223             K
             age                  7.6                   2.2           Gyr
       bb_source                   --                    --            --
       blackbody                   --                    --            --
             dec  -5.0413974999999995 

In [21]:
trap1.plot(integral=True)

Figure(id='4214', ...)

In [54]:
trap1.refs

{'parallax': '2018yCat.1345....0G',
 'photometry': {'2MASS.H': 'II/246/out',
  '2MASS.J': 'II/246/out',
  '2MASS.Ks': 'II/246/out',
  'Cousins.I': '2006AJ....132.1234C',
  'Cousins.R': '2006AJ....132.1234C',
  'Johnson.V': '2006AJ....132.1234C',
  'WISE.W1': 'II/328/allwise',
  'WISE.W2': 'II/328/allwise',
  'WISE.W3': 'II/328/allwise',
  'WISE.W4': 'II/328/allwise'},
 'spectra': {'Trappist-1_NIR.fits': 'Filippazzo et al. (2015)',
  'Trappist-1_OPT.fits': 'Cruz et al. (2003)'},
 'spectral_type': '2007AJ....133.2258S'}

### SED for Gl 752B

In [13]:
gl = SED('Gl 752B', method_list=['find_2MASS'])
gl.add_spectrum_file('Gl752B_OPT.txt', snr=10, wave_units=q.AA, flux_units=q.erg/q.s/q.cm**2/q.AA)
gl.add_spectrum_file('Gl752B_NIR.fits', wave_units=q.um, flux_units=q.erg/q.s/q.cm**2/q.AA)
gl.fit_spectral_type()
gl.fit_modelgrid(BTSettl())
gl.teff_from_age()
gl.logg_from_age()
gl.mass_from_age(mass_units=q.Mjup)
gl.plot(integral=True)

Setting parallax to (<Quantity 168.962 mas>, <Quantity 0.1299 mas>) and distance to (<Quantity 5.92 pc>, <Quantity 0. pc>, <Quantity 0. pc>).
Setting radius to (<Quantity 0.12250478 solRad>, <Quantity 0.00160632 solRad>)
1 record for Gl 752B found in Simbad.
Setting name to Gl 752B and sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (289.24004292, 5.15044111)>
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)
1 record found in 2MASS.
Best fit:  Opt:M7V
Best fit:  0.0/2600.0/5.5/0.0


Figure(id='2663', ...)

# Make a Catalog

In [14]:
cat = Catalog('M8 Stars')
cat.add_SED(trap1)
cat.add_SED(gl)
cat.results

Successfully added SED 'trappist-1'
Successfully added SED 'Gl 752B'


name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,Johnson.V,Johnson.V_unc,M_Johnson.V,M_Johnson.V_unc,Cousins.R,Cousins.R_unc,M_Cousins.R,M_Cousins.R_unc,Cousins.I,Cousins.I_unc,M_Cousins.I,M_Cousins.I_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc,WISE.W4,WISE.W4_unc,M_WISE.W4,M_WISE.W4_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
trappist-1,7.6,2.2,12.14,0.12,82.4,0.8,0.121,0.003,68.0,0.5,M8V,Opt:M8V,None,0.0010043595684692264,1.1587359896461991e-10,6.624516308416031e-13,13.358,0.006,2.0433032418319025e+30,4.204995989786862e+28,-3.273,0.009,12.937,0.012,5.289827450980392,0.003472445820435155,0.09005784313725489,0.000838699690402428,2522.0,33.0,2621.7382352941177,16.754426619132573,None,<sedkit.sed.SED object at 0x125cabbe0>,18.8,0.082,18.38,0.083,16.47,0.065,16.05,0.066,14.02,0.115,13.6,0.116,11.35,0.022,10.93,0.024,10.72,0.021,10.3,0.024,10.3,0.023,9.875,0.025,10.07,0.023,9.65,0.025,9.805,0.02,9.38,0.023,9.51,0.044,9.086,0.045,8.56,nan,8.14,nan
Gl 752B,6.0,4.0,5.92,0.0,168.962,0.1298999935388565,0.12250477577816277,0.0016063213337554596,68.0,0.5,M8V,Opt:M7V,None,6.380146078299731e-05,4.4901026798570266e-10,2.7367645697915413e-12,11.887,0.007,1.8828279666338387e+30,1.1476033484072167e+28,-3.308,0.003,13.025,0.007,5.29513963161022,0.0037596223674647433,0.08897207367795604,0.00017825311942959443,2455.0,16.0,2588.2307783719552,5.799286987522464,None,<sedkit.sed.SED object at 0x11dbefb70>,--,--,--,--,--,--,--,--,--,--,--,--,9.91,0.025,11.05,0.025,9.23,0.026,10.37,0.026,8.766,0.022,9.91,0.022,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [15]:
show(cat.plot_SEDs(['trappist-1', 'Gl 752B'], normalized=True, best_fit=False))

## Catalog from a list

In [16]:
# Make a catalog
catfile = Catalog('M Dwarfs')

# Add all the sources
catfile.from_file('sources.txt')

Generating SEDs for 25 sources from sources.txt
Successfully added SED 'trappist-1'
Successfully added SED 'LHS 102'
Successfully added SED 'LP 704-43'
Successfully added SED 'LHS 1'
Successfully added SED 'LHS 2'
Successfully added SED 'LHS 1047AB'
Successfully added SED 'LHS 3'
Successfully added SED 'LHS 4'
Successfully added SED 'LP 944-20'
Successfully added SED 'LHS 2065'
Successfully added SED 'LHS 1070 ABC'
Successfully added SED 'LHS 1075'
Successfully added SED 'LP 465-27'
Successfully added SED 'LHS 114 AB'
Successfully added SED 'LHS 115'
Successfully added SED 'LP 405-39'
Successfully added SED 'LP 406-29'
Successfully added SED 'LHS 131'
Successfully added SED 'LHS 1179'
Successfully added SED 'LHS 132'
Successfully added SED 'LHS 1183'
Successfully added SED 'LHS 134'
Successfully added SED 'LHS 1208'
Successfully added SED 'LP 467-16'
Successfully added SED 'LHS 138'


In [19]:
catfile.remove_SED('LHS 4')

In [20]:
col_mag = catfile.plot('spectral_type', 'Lbol_sun', legend='M dwarfs', order=2, identify=['trappist-1'])
show(col_mag)

Successfully added SED 'trappist-1'


In [21]:
BC_J = catfile.plot('2MASS.J-2MASS.H', 'M_2MASS.J', legend='M dwarfs', identify=['trappist-1'])
show(BC_J)

Successfully added SED 'trappist-1'
